# 电池数据异常检测
### 此notebook为异常检测比赛的参考DEMO，包括数据划分、模型训练和结果检测等

In [3]:
import numpy as np
from glob import glob
import torch
import pandas as pd
import random
from tqdm import tqdm
from collections import OrderedDict
from sklearn import metrics
from sklearn.metrics import auc
from pyod.models.deep_svdd import DeepSVDD
from pyod.models.auto_encoder_torch import AutoEncoder, check_array, inner_autoencoder, check_is_fitted, \
    pairwise_distances_no_broadcast
from glob import glob

2022-09-15 17:58:21.258864: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 加载训练集的pkl文件

训练集的label存放在pkl里面，可以通过它并区分正常片段和异常片段  
注意需要输入训练集对应的路径

In [4]:
data_path='/home/min/vloong-Anomaly-detection/repo_template/train'#存放数据的路径
pkl_files = glob(data_path+'/*.pkl')

In [5]:
ind_pkl_files = []#存放标签为0的文件
ood_pkl_files = []#存放标签为1的文件
for each_path in tqdm(pkl_files):
    this_pkl_file = torch.load(each_path)#下载pkl文件
    if this_pkl_file[1]['label'] == '00':
        ind_pkl_files.append(each_path)
    else:
        ood_pkl_files.append(each_path)

random.seed(0)
#排序并打乱存放车辆序号的集合
random.shuffle(ind_pkl_files)
random.shuffle(ood_pkl_files)

100%|███████████████████████████████████| 28389/28389 [00:09<00:00, 2972.86it/s]


### 划分训练集和测试集  

参赛选手可以根据需求自由化分数据集

这里选取训练集中正常片段的1/4作为训练集，正常片段的剩余3/4和异常片段作为测试集

In [8]:
train_pkl_files=[]
for i in range(len(ind_pkl_files)//4):
    train_pkl_files.append(ind_pkl_files[i])

In [9]:
test_pkl_files=[]
for j in range(len(ind_pkl_files)//4,len(ind_pkl_files)):
    test_pkl_files.append(ind_pkl_files[j])
for item in ood_pkl_files:
    test_pkl_files.append(item)

### 定义加载函数，并对数据进行正则化  

In [10]:
def  load_data(pkl_list,label=True):
    '''
    输入pkl的列表，进行文件加载
    label=True用来加载训练集
    label=False用来加载真正的测试集，真正的测试集无标签
    '''
    X = []
    y = []
    

    for  each_pkl in pkl_list:
        item = torch.load(each_pkl)
        # 此处选取的是每个滑窗的最后一条数据，仅供参考，可以选择其他的方法，比如均值或者其他处理时序数据的网络
        # 此处选取了前7个特征，可以需求选取特征数量
        X.append(item[0][:,0:7][-1])
        if label:
            y.append(int(item[1]['label'][0]))
    X = np.vstack(X)
    if label:
        y = np.vstack(y)
    return X, y

In [11]:
X_train,y_train=load_data(train_pkl_files)

In [12]:
X_test,y_test=load_data(test_pkl_files)

In [13]:
_mean = np.mean(X_train, axis=0)
_std = np.std(X_train, axis=0)
X_train = (X_train - _mean) / (_std + 1e-4)
X_test = (X_test - _mean) / (_std + 1e-4)

### 定义DataLoader数据集

In [14]:
class PyODDataset(torch.utils.data.Dataset):
    """PyOD Dataset class for PyTorch Dataloader
    """

    def __init__(self, X, y=None, mean=None, std=None):
        super(PyODDataset, self).__init__()
        self.X = X
        self.mean = mean
        self.std = std

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()#将tensor类型转换列表格式
        sample = self.X[idx, :]

        # if self.mean.any():
        #     sample = (sample - self.mean) / (self.std + 1e-5)
        #torch.from_numpy()将numpy类型转换为tensor类型
        return torch.from_numpy(sample), idx

### 定义AutoEncoder类

In [15]:
class Car_AutoEncoder(AutoEncoder):
    
    '''
    使用autoencoder 来进行模型的训练，默认采用无监督的训练方式
    '''
    
    def fit(self, X, y=None):
        # validate inputs X and y (optional)
        X = check_array(X)
        self._set_n_classes(y)

        n_samples, n_features = X.shape[0], X.shape[1] #获取样本个数和特征个数

        # 是否进行预处理操作
        if self.preprocessing:
            self.mean, self.std = np.mean(X, axis=0), np.std(X, axis=0)
            train_set = PyODDataset(X=X, mean=self.mean, std=self.std)

        else:
            train_set = PyODDataset(X=X)
        #构建数据生成器
        train_loader = torch.utils.data.DataLoader(train_set,
                                                   batch_size=self.batch_size,
                                                   shuffle=False)

        # initialize the model ,初始化模型
        #hidden_neurons列表，可选（默认值为[64， 32]）每个隐藏层的神经元数。因此，该网络的结构为[n_features，64，32，32，64，n_features]
        #hidden_activationstr，可选（默认值='relu'）用于隐藏层的激活函数。所有隐藏层都强制使用相同类型的激活
        #batch_norm布尔值，可选（默认值为 True）是否应用批量规范化。
        #dropout_rate浮点数 （0.， 1），可选（默认值 = 0.2）要跨所有层使用的分级。
        
        self.model = inner_autoencoder(
            n_features=n_features,
            hidden_neurons=self.hidden_neurons,
            dropout_rate=self.dropout_rate,
            batch_norm=self.batch_norm,
            hidden_activation=self.hidden_activation)

        #将model放入device中
        self.model = self.model.to(self.device)

        # 训练自动编码器以找到最佳编码器
        self._train_autoencoder(train_loader)

        self.model.load_state_dict(self.best_model_dict)
        self.decision_scores_ = self.decision_function(X)#获得输入样本的异常得分
        
        self._process_decision_scores()  
        return self

    def decision_function(self, X): 
        """使用拟合的检测器预测X的原始异常分数。

            输入样本的异常分数是基于不同的检测器算法。为保持一致性，离群值分配为异常分数越大的。
        Parameters
        ----------
        X : numpy array of shape (n_samples, n_features)
            形状的numpy数组（n_samples，n_features）训练输入样本。仅接受稀疏矩阵，如果它们由基础估计器支持。

        Returns
        -------
        anomaly_scores : numpy array of shape (n_samples,)
            形状的numpy数组（n_samples，）输入样本的异常得分。
        """
        #对估算器执行is_fitted验证。通过验证是否存在拟合属性（以下划线结尾）来检查估计量是否拟合，否则通过给定消息引发NotFittedError。此实用程序旨在由估计器本身在内部使用，通常在其自己的预测/变换方法中使用。
        check_is_fitted(self, ['model', 'best_model_dict'])
        # X = check_array(X)

        # note the shuffle may be true but should be False
        if self.preprocessing:
            dataset = PyODDataset(X=X, mean=self.mean, std=self.std)
        else:
            dataset = PyODDataset(X=X)

        dataloader = torch.utils.data.DataLoader(dataset,
                                                 batch_size=self.batch_size,
                                                 shuffle=False) #要设置为False
        # enable the evaluation mode
        self.model.eval()

        # construct the vector for holding the reconstruction error
        outlier_scores = np.zeros([X.shape[0], ])#形状为（X.shape[0],)
        with torch.no_grad():
            for data, data_idx in dataloader:
                data_cuda = data.to(self.device).float()
                # this is the outlier score
                outlier_scores[data_idx] = pairwise_distances_no_broadcast(
                    data, self.model(data_cuda).cpu().numpy())

        return outlier_scores

### 定义训练和评估函数

In [16]:
# 为了便于复现这里固定了随机种子
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    
def train(X_train,X_test,hidden_neurons,learning_rate,epochs,batch_size,contamination,drop_out,hidden_activation):
    same_seeds(42)
    clf_name = 'auto_encoder'
    clf = Car_AutoEncoder(hidden_neurons=hidden_neurons,  batch_size=batch_size, epochs=epochs,learning_rate=learning_rate,
                                    dropout_rate=drop_out,contamination=contamination,hidden_activation=hidden_activation)
    clf.fit(X_train)

    y_train_pred = clf.labels_ # 返回训练数据上的分类标签 (0: 正常值, 1: 异常值) # binary labels (0: inliers, 1: outliers)
    y_train_scores = clf.decision_scores_  # 返回训练数据上的异常值 (分值越大越异常)# raw outlier scores
    y_test_pred = clf.predict(X_test) # 返回未知数据上的分类标签 (0: 正常值, 1: 异常值) # outlier labels (0 or 1)
    y_test_scores = clf.decision_function(X_test)   #返回未知数据上的异常值 (分值越大越异常) # outlier scores
    return clf, y_test_scores

def evaluate(label,score):
    fpr, tpr, thresholds = metrics.roc_curve(label, score, pos_label=1)
    AUC = auc(fpr, tpr)
    return AUC

### 模型的训练和AUC计算

In [46]:
# 定义合适的参数
hidden_neurons=[ 64,128,128,64]
learning_rate=0.02
epochs=50
batch_size=640
contamination=0.005
drop_out=0.2
hidden_activation='sigmoid'

In [47]:
same_seeds(42)
clf = Car_AutoEncoder(hidden_neurons=hidden_neurons,  batch_size=batch_size, epochs=epochs,learning_rate=learning_rate,
                                     dropout_rate=drop_out,contamination=contamination,hidden_activation=hidden_activation)
clf.fit(X_train)
y_test_pred = clf.predict(X_test) # 返回未知数据上的分类标签 (0: 正常值, 1: 异常值) # outlier labels (0 or 1)
y_test_scores = clf.decision_function(X_test)   #返回未知数据上的异常值 (分值越大越异常) # outlier scores


epoch 0: training loss 0.83499715924263 
epoch 1: training loss 0.508006152510643 
epoch 2: training loss 0.478377440571785 
epoch 3: training loss 0.46995710730552676 
epoch 4: training loss 0.455228129029274 
epoch 5: training loss 0.44920630753040314 
epoch 6: training loss 0.427818164229393 
epoch 7: training loss 0.39976899027824403 
epoch 8: training loss 0.38707375824451445 
epoch 9: training loss 0.37535335719585416 
epoch 10: training loss 0.36480623185634614 
epoch 11: training loss 0.3499638170003891 
epoch 12: training loss 0.3335436999797821 
epoch 13: training loss 0.3083561807870865 
epoch 14: training loss 0.302016681432724 
epoch 15: training loss 0.29258138239383696 
epoch 16: training loss 0.2819209784269333 
epoch 17: training loss 0.2840063750743866 
epoch 18: training loss 0.2781553566455841 
epoch 19: training loss 0.27419638335704805 
epoch 20: training loss 0.2750765323638916 
epoch 21: training loss 0.2753778547048569 
epoch 22: training loss 0.270931942760944

In [48]:
AUC1=evaluate(y_test,y_test_scores)
AUC1

0.6887876776808659

### 模型的保存、加载和提交文件生成  

需要输入测试数据的路径

保存和加载模型

In [27]:
torch.save(clf,"clf1.torch")

In [28]:
clf=torch.load("clf1.torch")
clf

Car_AutoEncoder(batch_norm=True, batch_size=640, contamination=0.005,
        device=device(type='cpu'), dropout_rate=0.2, epochs=15,
        hidden_activation='sigmoid', hidden_neurons=[32, 64, 64, 32],
        learning_rate=0.03, loss_fn=MSELoss(), preprocessing=True,
        weight_decay=1e-05)

测试集验证

In [31]:
data_path3='/home/min/vloong-Anomaly-detection/repo_template/test_A'
test1_files = glob(data_path3+'/*.pkl')

In [32]:
X_val,_=load_data(test1_files,label=False)
_mean = np.mean(X_val, axis=0)
_std = np.std(X_val, axis=0)
X_val = (X_val - _mean) / (_std + 1e-4)
y_val_pred = clf.predict(X_val) # 返回未知数据上的分类标签 (0: 正常值, 1: 异常值) # outlier labels (0 or 1)
y_val_scores = clf.decision_function(X_val)   #返回未知数据上的异常值 (分值越大越异常) # outlier scores

In [33]:
import platform
#记录文件名和对应的异常得分
predict_result={}
for i in tqdm(range(len(test1_files))):
    file=test1_files[i]
    #如果是window系统：
    if platform.system().lower() == 'windows':
        name=file.split('\\')[-1]
    #如果是linux系统
    elif platform.system().lower() == 'linux':
        name=file.split('/')[-1]
    predict_result[name]=y_val_scores[i]

100%|███████████████████████████████████| 6234/6234 [00:00<00:00, 354890.82it/s]


In [34]:
predict_score=pd.DataFrame(list(predict_result.items()),columns=['file_name','score'])#列名必须为这俩个
predict_score

,file_name,score
0,6387.pkl,0.927133
1,3382.pkl,0.758415
2,5525.pkl,0.621016
3,5468.pkl,0.554446
4,8606.pkl,0.778990
...,...,...
6229,2225.pkl,0.793437
6230,3846.pkl,0.461130
6231,9864.pkl,0.818715
6232,4059.pkl,0.409528


生成csv文件

In [36]:
predict_score.to_csv('submision.csv',index = False) #保存为比赛要求的csv文件

In [37]:
predict_score[0:10]

,file_name,score
0,6387.pkl,0.927133
1,3382.pkl,0.758415
2,5525.pkl,0.621016
3,5468.pkl,0.554446
4,8606.pkl,0.778990
5,2511.pkl,0.848401
6,5153.pkl,1.217900
7,4974.pkl,0.637358
8,9962.pkl,0.416316
9,3795.pkl,0.619432
